# SETUP

In [1]:
import pandas as pd

In [89]:
data_raw = pd.read_pickle('/Users/quartz/data/collabo/solar/main_data_v2.pkl')
data_raw.tail()

,날짜,충전시간발전량,일출시간,일몰시간,일일 강수량,습도 평균,풍속 평균,기온 평균,풍향,풍정합(100m),평균 증기압(hPa),평균 현지기압(hPa),최고 해면기압(hPa),평균 해면기압(hPa),가조시간(hr),합계 일조 시간(hr),일일 일조량,자외선B,운량,미세먼지
376,170712,2392.2,530,1946,0.0,88.4375,1.63750,23.66875,290,713,29.3,1006.5,1012.2,1010.5,14.3,9.1,19.60,10.0,21.878049,0.202
377,170713,1625.2,530,1946,0.0,90.7500,1.26875,24.56250,270,705,31.3,1004.5,1010.0,1008.5,14.3,7.1,17.56,9.5,22.086957,0.185
378,170714,1764.5,531,1946,0.0,93.3125,0.73750,25.46875,270,1218,34.0,1004.8,1010.2,1008.8,14.3,5.4,12.90,5.4,17.785714,0.184
379,170715,937.2,531,1945,0.0,92.5625,0.54375,24.88750,270,1603,35.1,1007.2,1013.1,1011.2,14.2,1.1,3.51,4.9,17.266667,0.221
380,170716,1945.4,532,1945,0.0,93.6875,0.86875,25.54375,200,1982,35.7,1009.0,1014.1,1013.0,14.2,9.1,13.66,7.0,12.567568,0.094


# 플랜

- 전처리
    - 스케일링
    - 데이터셋
- 모델링
    - 선형

In [90]:
data_raw.dtypes

날짜                int64
충전시간발전량         float64
일출시간              int64
일몰시간              int64
일일 강수량          float64
습도 평균           float64
풍속 평균           float64
기온 평균           float64
풍향                int64
풍정합(100m)         int64
평균 증기압(hPa)     float64
평균 현지기압(hPa)    float64
최고 해면기압(hPa)    float64
평균 해면기압(hPa)    float64
가조시간(hr)        float64
합계 일조 시간(hr)    float64
일일 일조량          float64
자외선B            float64
운량              float64
미세먼지            float64
dtype: object

---

# 전처리

### 데이터셋 분리

In [91]:
from sklearn.model_selection import train_test_split

y = data_raw.iloc[:, 1:2]
X = data_raw.drop(['날짜', '충전시간발전량'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((304, 18), (77, 18), (304, 1), (77, 1))

### 스케일링

In [92]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

y_train_sc = scaler.fit_transform(y_train)
y_test_sc = scaler.transform(y_test)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [93]:
X_train_df = pd.DataFrame(data=X_train_sc, columns=X_train.columns)
X_test_df = pd.DataFrame(data=X_test_sc, columns=X_test.columns)
y_train_df = pd.DataFrame(data=y_train_sc, columns=['발전량'])
y_test_df = pd.DataFrame(data=y_test_sc, columns=['발전량'])

In [102]:
data_raw.iloc[304:].head(10)

,날짜,충전시간발전량,일출시간,일몰시간,일일 강수량,습도 평균,풍속 평균,기온 평균,풍향,풍정합(100m),평균 증기압(hPa),평균 현지기압(hPa),최고 해면기압(hPa),평균 해면기압(hPa),가조시간(hr),합계 일조 시간(hr),일일 일조량,자외선B,운량,미세먼지
304,170501,2466.5,543,1917,0.0,73.0000,2.73750,15.47500,140,4221,15.4,1012.7,1020.4,1016.8,13.6,10.8,20.94,0.0,75.625000,0.178
305,170502,1975.5,542,1917,0.0,50.0625,3.25000,15.93125,160,4218,10.3,1017.3,1022.7,1021.4,13.6,6.9,19.90,1.1,30.958333,0.164
306,170503,2405.1,541,1918,0.0,76.1250,1.52500,15.53750,180,2333,14.0,1017.1,1022.9,1021.2,13.6,11.2,21.57,7.6,37.104167,0.196
307,170504,1890.3,540,1919,4.6,86.0000,1.80625,16.83125,140,2178,16.9,1012.4,1020.0,1016.5,13.7,9.0,17.80,8.0,37.187500,0.195
308,170505,213.5,539,1920,1.2,80.8750,3.03750,16.50000,360,1810,18.9,1010.2,1015.9,1014.3,13.7,0.0,4.60,0.0,23.065217,0.224
309,170506,2022.3,538,1921,0.0,48.6875,3.19375,15.56875,360,2155,6.8,1012.7,1018.5,1016.9,13.7,8.2,19.65,6.1,163.458333,0.203
310,170507,2448.1,537,1921,0.0,76.1875,2.76875,15.00000,320,2649,12.5,1012.3,1018.3,1016.4,13.7,11.3,22.94,9.3,185.979167,0.168
311,170508,1266.0,536,1922,1.0,53.2500,1.82500,16.01250,160,1365,12.1,1010.7,1016.8,1014.9,13.8,4.8,12.61,5.4,176.166667,0.090
312,170509,66.3,535,1923,3.1,90.3125,2.23125,14.17500,140,3883,14.8,1005.8,1014.7,1010.0,13.8,0.0,4.25,7.6,33.531915,0.184
313,170510,1175.2,535,1924,0.0,85.6875,1.80000,13.43750,320,2482,15.9,1007.6,1015.1,1011.8,13.8,3.2,14.50,10.0,27.958333,0.187


# 모델링

### 성능 지표

- 예측 모델 평가에 사용할 주된 지표는 MAE(mean absolute error) 값으로 절 대 평균 오차를 의미한다. 즉 실측값으로 부터의 ±오차의 평균을 나타낸다고 볼 수 있다. 그리고 MAPE(mean absolute percentage error)지표를 사용하였다. MAPE는 평균 절대 오차율로 실측값으로부터 벗어난 예측값에 대한 비율의 평균을 의미한다. 하지만 실측값이 0인 경우 MAPE 값이 무한대가 된다. 이러 한 문제로 기존 태양광 발전량 예측 연구에서는 MAPE 값을 하루단위 총 발 전량을 사용하였다. 본 연구에서도 MAPE 값을 하루단위 총 발전량을 기준으 로 하였다. MAE와 MAPE를 산출하는 공식은 다음과 같다.

### 선형 회귀

In [94]:
# score function

def mse_score(y_pred, y_test):
    result = np.mean(np.power(y_pred - y_test, 2))
    return result

def mape_score(y_pred, y_test):
    result = np.mean(abs((y_test - y_pred) / y_test))
    return result

### scikit-learn

In [95]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
linear_model.fit(X_train_df, y_train_df)
y_pred = linear_model.predict(X_test_df)

print(mape_score(y_pred, y_test_sc), mse_score(y_pred, y_test_sc))

# coef_

result_df = pd.DataFrame(columns=['feature', 'coef'])
result_df['feature'] = X_train_df.columns
result_df['coef'] = linear_model.coef_[0]
result_df 

# 1 main_data : 1.9163773879379122 0.006170525138753231
# 2 main_data_v2 : 0.22536565269425068 0.006355496015255523

0.22536565269425068 0.006355496015255523


,feature,coef
0,일출시간,0.015025
1,일몰시간,-0.004146
2,일일 강수량,-0.025527
3,습도 평균,-0.161546
4,풍속 평균,-0.056067
5,기온 평균,-0.008862
6,풍향,-0.002130
7,풍정합(100m),0.064376
8,평균 증기압(hPa),0.241721
9,평균 현지기압(hPa),-1.480186


### statsmodels

In [97]:
import statsmodels.api as sm

model = sm.OLS(y_train_df, X_train_df)
result = model.fit()

y_pred = result.predict(X_test_df)
print(mape_score(y_pred.values, y_test_sc), mse_score(y_pred.values, y_test_sc))

# 1 main_data : 14.440597100253086 0.15544080868189367
# 2 main_data_v2 : 1.7318974325224947 0.14596765335918824

1.7318974325224947 0.14596765335918824


In [98]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    발전량   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     1275.
Date:                Fri, 04 Jan 2019   Prob (F-statistic):          4.08e-261
Time:                        16:05:40   Log-Likelihood:                 369.35
No. Observations:                 304   AIC:                            -702.7
Df Residuals:                     286   BIC:                            -635.8
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
일출시간             0.0298      0.088      0.337   

### 노트

- 습도 평균, 평균 증기압, 가조시간, 합계 일조시간, 일일 일조량, 운량 등이 유의한 특성이라 할 수 있다.
- 특히 합계 일조 시간, 일일 일조량(강진)의 상관계수는 매우 높다.
> 논문(이강혁, "서포트 벡터 회귀를 이용한 24시간 후의 태양광 발전량 예측", 2016)의 내용과 일치한다.

- main_data_v2를 사용하면 퍼포먼스는 늘어나지만 조건수는 50에서 2500으로 확 높아진다

### 교차 검증

In [56]:
from sklearn.model_selection import cross_val_score

In [99]:
result = cross_val_score(linear_model, X=X_train_df, y=y_train_df, scoring='neg_mean_squared_error', cv=5, verbose=1)
np.mean(-result)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


0.005995050973622899